In [1]:
from Dental_Tool.Data_processing import *
from Dental_Tool.Dental_Model import *
from Dental_Tool.Process_results import *
from Dental_Tool.Dataloader import *
from Dental_Tool.KFold_v3 import *

Using TensorFlow backend.


In [2]:
directory = [ 
                "Dental_Data/PBL/10_20200901", 
                "Dental_Data/PBL/10_20200901_Flip", 
                "Dental_Data/PBL/10_clahe_20200901", 
                "Dental_Data/PBL/10_clahe_20200901_Flip"
            ]

directory = [ i + "/mapping.json" for i in directory]
argscale_num = len(directory) * 20
data = load_json(directory, interdental=False)
dataset = json_2_dataframe_PBL(data)
dataset = dataset[dataset.Class == 2]



In [3]:
image_size = 128
input_shape = (image_size, image_size, 1)
batch_size = 32
kernel_size = 3
filters = 16
latent_dim = 64 # 隐变量取2维只是为了方便后面画图
epochs = 30
num_classes = classes = 4
tooth_type = 4

In [4]:
x_in      = Input(shape=input_shape, name='encoder_input')
condition = Input(shape=(tooth_type, ), name='class_labels')
y_labels  = Dense(image_size * image_size)(condition)
y_labels  = Reshape((image_size, image_size, 1))(y_labels)

# x = x_in
x = concatenate([x_in, y_labels])

deep_layer = 3

for i in range(deep_layer):
    filters *= 2
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               activation='relu',
               strides=2,
               padding='same')(x)

# 备份当前shape，等下构建decoder的时候要用
shape = K.int_shape(x)

x = Flatten()(x)
x = Dense(latent_dim, activation='relu')(x)
# # 算p(Z|X)的均值和方差
z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)

# y_input = Input(shape=(num_classes,)) # 输入类别

# yh = Dense(latent_dim)(y) # 这里就是直接构建每个类别的均值



# 重参数技巧
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=K.shape(z_mean))
    return z_mean + K.exp(z_log_var / 2) * epsilon

# 重参数层，相当于给输入加入噪声
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# 解码层，也就是生成器部分
# 先搭建为一个独立的模型，然后再调用模型
latent_inputs = Input(shape=(latent_dim,))
z_input = concatenate([latent_inputs, condition])


# label_inputs = Input(shape=(num_classes,))
# x = Concatenate()([latent_inputs])
x = Dense(shape[1] * shape[2] * shape[3], activation='relu')(z_input)
x = Reshape((shape[1], shape[2], shape[3]))(x)

for i in range(deep_layer):
    x = Conv2DTranspose(filters=filters,
                        kernel_size=kernel_size,
                        activation='relu',
                        strides=2,
                        padding='same')(x)
    filters //= 2

outputs = Conv2DTranspose(filters=1,
                          kernel_size=kernel_size,
                          activation='sigmoid',
                          padding='same')(x)

# 搭建为一个独立的模型
decoder = Model([latent_inputs, condition], outputs)

x_out = decoder([z, condition])

# 建立模型
vae = Model([x_in, condition] , x_out)

# xent_loss是重构loss，kl_loss是KL loss
xent_loss = K.sum(K.binary_crossentropy(x_in, x_out), axis=-1)

# 只需要修改K.square(z_mean)为K.square(z_mean - yh)，也就是让隐变量向类内均值看齐
kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
vae_loss = K.mean(xent_loss + kl_loss)

# # xent_loss是重构loss，kl_loss是KL loss
# xent_loss = K.sum(K.binary_crossentropy(x_in, x_out), axis=[1, 2, 3])
# kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
# vae_loss = K.mean(xent_loss + kl_loss)

# add_loss是新增的方法，用于更灵活地添加各种loss
vae.add_loss(vae_loss)
vae.compile(optimizer='rmsprop')
vae.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
class_labels (InputLayer)       (None, 4)            0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 16384)        81920       class_labels[0][0]               
__________________________________________________________________________________________________
encoder_input (InputLayer)      (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 128, 128, 1)  0           dense_1[0][0]                    
____________________________________________________________________________________________

C:\Users\Lab620\Anaconda3\envs\lawrence\lib\site-packages\keras\engine\training_utils.py:819: UserWarning: Output model_1 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to model_1.
  'be expecting any data to be passed to {0}.'.format(name))


In [5]:
def sampling(args: tuple):
        # we grab the variables from the tuple
        z_mean, z_log_var = args
        epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0.,
                                  stddev=epsilon_std)
        return z_mean + K.exp(z_log_var / 2) * epsilon

In [6]:
def load_images(path_list, resize):
        X = []
        for path in tqdm(path_list):
                image = cv2.imread(path, 0)
                image = cv2.resize(image, resize)
                image = image.astype("float32") / 255.0
#                 image = image - np.mean(image)
                image = np.expand_dims(image, axis=2)
                X.append(image)
        return np.array(X)

dataset_size = len(dataset)

dataset = shuffle(dataset).reset_index(drop=True)
train_idx = int(dataset_size * 0.6)
valid_idx = int(dataset_size * 0.2)

train = dataset.iloc[:train_idx]
valid = dataset.iloc[train_idx: train_idx + valid_idx]
test  = dataset.iloc[train_idx + valid_idx: ]

classes = 4
x_train, y_train = load_images(train["Path"], (image_size, image_size)), to_categorical(train["tooth_type"], classes)
x_valid, y_valid = load_images(valid["Path"], (image_size, image_size)), to_categorical(valid["tooth_type"], classes)
x_test, y_test   = load_images(test["Path"], (image_size, image_size)), to_categorical(test["tooth_type"], classes)



print(x_train.shape)
print(y_train.shape)

# for train, valid, test, train_gen, valid_gen, test_gen in \
#     K_Fold_balance_data_generator(dataset, argscale_num, batch_size=32, k_fold_num=5):
        
#         x_train, y_train = load_images(train["Path"], (image_size, image_size)), to_categorical(train["Class"], classes)
#         x_valid, y_valid = load_images(valid["Path"], (image_size, image_size)), to_categorical(valid["Class"], classes)
#         x_test, y_test = load_images(test["Path"], (image_size, image_size)), to_categorical(test["Class"], classes)
        
#         print(x_train.shape), print(y_train.shape)
#         break


(19248, 128, 128, 1)
(19248, 4)


In [8]:
vae.fit([x_train, y_train], 
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=([x_valid, y_valid], None))

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 2 array(s), but instead got the following list of 1 arrays: [array([[[[0.14509805],
         [0.6901961 ],
         [0.7254902 ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.2627451 ],
         [0.6509804 ],...